In [1]:
!pip install pandas
!pip install scikit-learn
!pip install nltk
!pip install tiktoken

In [2]:
import pandas as pd
import tiktoken
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

def tokenizer(X):
    enc = tiktoken.encoding_for_model("gpt-4")
    stop_words = set(stopwords.words('english'))
    num_tokens = 0
    num_sentences = 0
    tokenized_documents = []
    for text in X:
        tokenized_document = enc.encode(text)

        filtered_tokens = [token for token in tokenized_document if token not in stop_words]

        num_tokens += len(filtered_tokens)
        num_sentences += text.count('.') + text.count('!') + text.count('?')
        
        tokenized_documents.append(filtered_tokens)

    print("Number of tokens: ", num_tokens)
    print("Number of sentences: ", num_sentences)
    
    return tokenized_documents


import sklearn.feature_extraction.text as sk_text

def vectorizer(tokenized_documents):
    # Conversion des bytes en chaînes de caractères si nécessaire
    tokenized_documents_str = []
    for document in tokenized_documents:
        if isinstance(document, list):
            # Convertir les bytes en chaînes de caractères
            tokenized_document_str = [token.decode('utf-8') if isinstance(token, bytes) else str(token) for token in document]
            tokenized_documents_str.append(tokenized_document_str)
        else:
            # Si le document n'est pas une liste, le convertir en liste de chaîne de caractères
            tokenized_documents_str.append([str(document)])

    # Initialiser le vectoriseur
    vectorizer = sk_text.CountVectorizer(lowercase=False, preprocessor=None, tokenizer=lambda x: x)
    
    # Adapter et transformer les données
    X = vectorizer.fit_transform(tokenized_documents_str)
    
    return X, vectorizer

[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# NLTK TOKENIZER AND TF-IDF VECTORIZER

In [3]:
import sys
# sys.path.append('../')
import numpy as np
import pandas as pd
import sklearn as sk

In [4]:
data = pd.read_csv('../../_data/Reviews.csv') # Loading the dataset
X, y = data['Text'], data['Score']

In [5]:
# FOR TESTING : only select first 20000 samples
# X, y = X[:20000], y[:20000]

In [6]:
tokenized_documents = tokenizer(X)

Number of tokens:  58325048
Number of sentences:  3661772


In [7]:
X, vect = vectorizer(tokenized_documents)

/opt/mamba/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [8]:
# most frequent words
print("Top 10 most frequent words in the dataset")
print(vect.get_feature_names_out()[:10])

# least frequent words
print("Top 10 least frequent words in the dataset")
print(vect.get_feature_names_out()[-10:])

Top 10 most frequent words in the dataset
['0' '1' '10' '100004' '100005' '10001' '100011' '100012' '100014'
 '100016']
Top 10 least frequent words in the dataset
['99975' '9998' '99981' '99984' '99985' '99986' '9999' '99990' '99992'
 '99994']


In [9]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

print(X_train[0])

(454763, 53351) (113691, 53351) (454763,) (113691,)
  (0, 34404)	1
  (0, 2531)	1
  (0, 23716)	1
  (0, 19965)	2
  (0, 22878)	1
  (0, 23523)	1
  (0, 899)	2
  (0, 5502)	1
  (0, 1672)	1
  (0, 30112)	1
  (0, 0)	1
  (0, 51388)	1
  (0, 39869)	1
  (0, 20900)	1
  (0, 932)	2
  (0, 52850)	1
  (0, 36993)	1
  (0, 14313)	1
  (0, 6293)	1
  (0, 24280)	1
  (0, 2731)	1
  (0, 656)	1
  (0, 1347)	1


# Recurrent Neural Network
### Model starts here

In [11]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn import metrics
import datetime
import os

2024-05-03 23:41:39.820091: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 23:41:39.856078: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 23:41:40.263245: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
def batch_generator(X, y, batch_size=32, num_classes=None, timesteps=1):
    num_samples = X.shape[0]
    features_per_timestep = X.shape[1] // timesteps
    while True:
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            X_batch = X[start:end].toarray()  # Convert only this slice to dense
            X_batch = X_batch.reshape(-1, timesteps, features_per_timestep)  # Reshape for LSTM
            y_batch = to_categorical(y[start:end], num_classes=num_classes)
            yield (X_batch, y_batch)


# Setup the model
model = Sequential([
    LSTM(256, input_shape=(None, X_train.shape[1]), activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(128, activation='relu'),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(np.max(y_train) + 1, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model using a generator
train_generator = batch_generator(X_train, y_train, batch_size=128, num_classes=np.max(y_train) + 1, timesteps=1)
model.fit(train_generator, epochs=20, steps_per_epoch=int(np.ceil(X_train.shape[0] / 128)))

2024-05-03 23:41:41.514931: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78746 MB memory:  -> device: 0, name: NVIDIA H100 PCIe, pci bus id: 0000:b5:00.0, compute capability: 9.0
/opt/mamba/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20


I0000 00:00:1714779703.750870   19549 service.cc:145] XLA service 0x7f2ea0004fc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714779703.750905   19549 service.cc:153]   StreamExecutor device (0): NVIDIA H100 PCIe, Compute Capability 9.0
2024-05-03 23:41:43.835703: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-03 23:41:44.250421: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900
I0000 00:00:1714779707.179088   19745 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_59', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1714779708.425555   19769 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_58', 60 bytes spill stores, 60 bytes spill loads

I0000 00:00:1714779708.

  13/3553 ━━━━━━━━━━━━━━━━━━━━ 49s 14ms/step - accuracy: 0.5566 - loss: 1.7153

I0000 00:00:1714779711.798217   19549 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3551/3553 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7153 - loss: 0.7907

I0000 00:00:1714779778.049559   20282 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_59', 4 bytes spill stores, 4 bytes spill loads



3553/3553 ━━━━━━━━━━━━━━━━━━━━ 80s 20ms/step - accuracy: 0.7154 - loss: 0.7906
Epoch 2/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 61s 17ms/step - accuracy: 0.8052 - loss: 0.5266
Epoch 3/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 63s 18ms/step - accuracy: 0.8673 - loss: 0.3634
Epoch 4/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 59s 17ms/step - accuracy: 0.9067 - loss: 0.2591
Epoch 5/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 62s 18ms/step - accuracy: 0.9317 - loss: 0.1938
Epoch 6/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 64s 18ms/step - accuracy: 0.9483 - loss: 0.1482
Epoch 7/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 64s 18ms/step - accuracy: 0.9586 - loss: 0.1204
Epoch 8/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 65s 18ms/step - accuracy: 0.9652 - loss: 0.1014
Epoch 9/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 62s 17ms/step - accuracy: 0.9712 - loss: 0.0851
Epoch 10/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 62s 17ms/step - accuracy: 0.9753 - loss: 0.0738
Epoch 11/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 63s 18ms/step - accuracy: 0.9788 - loss: 0.0650
Epoch 12/20
3553/35

In [13]:
test_generator = batch_generator(X_test, y_test, batch_size=32, num_classes=np.max(y_train) + 1, timesteps=1)

In [14]:
# Evaluate the model
predictions_prob = model.predict(test_generator, steps=int(np.ceil(X_test.shape[0] / 32)))
predictions = np.argmax(predictions_prob, axis=1)

# Metrics
print(metrics.confusion_matrix(y_test, predictions))
print(metrics.classification_report(y_test, predictions))

3553/3553 ━━━━━━━━━━━━━━━━━━━━ 27s 7ms/step
[[ 7793   874   518   201   940]
 [  937  3062   896   282   678]
 [  565   617  4826  1071  1406]
 [  279   234  1108  9004  5498]
 [  660   333   847  3233 67829]]
              precision    recall  f1-score   support

           1       0.76      0.75      0.76     10326
           2       0.60      0.52      0.56      5855
           3       0.59      0.57      0.58      8485
           4       0.65      0.56      0.60     16123
           5       0.89      0.93      0.91     72902

    accuracy                           0.81    113691
   macro avg       0.70      0.67      0.68    113691
weighted avg       0.81      0.81      0.81    113691



In [15]:
# # Save the model with current date and time in model folder

# Create a folder named _models in the current directory
if not os.path.exists('_models'):
    os.makedirs('_models')
    
model.save(f'_models/config2_rnn_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.keras')

In [16]:
# test the model
test_sentences = [
    'This is a good product',
    'This is a bad product',
    'This is a product',
    'This is a very good product',
    'This is a very bad product',
    'That was bad'
]

for sentence in test_sentences:
    test_tokenized = tokenizer([sentence])
    test_vec = vect.transform(test_tokenized)
    test_vec_dense = test_vec.toarray()  # Convert sparse tensor to dense tensor
    test_vec_lstm = test_vec_dense.reshape(-1, 1, X.shape[1])
    result = model.predict(test_vec_lstm)
    predicted_class = np.argmax(result, axis=-1)
    predicted_score = predicted_class
    print(f"Test sentence: {sentence}")
    print(f"Predicted score: {predicted_score}")
    print()

Number of tokens:  5
Number of sentences:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 605ms/step
Test sentence: This is a good product
Predicted score: [1]

Number of tokens:  5
Number of sentences:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Test sentence: This is a bad product
Predicted score: [1]

Number of tokens:  4
Number of sentences:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Test sentence: This is a product
Predicted score: [1]

Number of tokens:  6
Number of sentences:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Test sentence: This is a very good product
Predicted score: [1]

Number of tokens:  6
Number of sentences:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Test sentence: This is a very bad product
Predicted score: [1]

Number of tokens:  3
Number of sentences:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Test sentence: That was bad
Predicted score: [1]

